# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


zsh:1: /Users/suvratbhatta/Desktop/Classes/DTSC3020/data_science/bin/pip: bad interpreter: /Users/suvratbhatta/Desktop/DTSC3020/data_science/bin/python3.13: no such file or directory
Dependencies installed.


### 2) Common Imports & Polite Headers

In [1]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# --- Q1 Implementation ---

def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    Uses pd.read_html to parse tables and selects one with sufficient columns.
    """
    # Parse all tables from HTML
    tables = pd.read_html(html)
    
    # Find first table with >= 3 columns
    for table in tables:
        if len(table.columns) >= 3:
            # Flatten multi-level column headers if present
            if isinstance(table.columns, pd.MultiIndex):
                table.columns = ['_'.join(map(str, col)).strip('_') for col in table.columns.values]
            return table
    
    raise ValueError("No table with >= 3 columns found")


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    """
    # Create a copy to avoid modifying original
    df_clean = df.copy()
    
    # Strip whitespace from all string columns
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].astype(str).str.strip()
    
    # Identify columns by name (case-insensitive matching)
    col_map = {}
    for col in df_clean.columns:
        col_lower = col.lower()
        if 'alpha-2' in col_lower or col_lower == 'alpha-2 code':
            col_map['Alpha-2'] = col
        elif 'alpha-3' in col_lower or col_lower == 'alpha-3 code':
            col_map['Alpha-3'] = col
        elif 'numeric' in col_lower or col_lower == 'numeric code':
            col_map['Numeric'] = col
        elif 'country' in col_lower:
            col_map['Country'] = col
    
    # Convert Alpha-2 and Alpha-3 to UPPERCASE
    if 'Alpha-2' in col_map:
        df_clean[col_map['Alpha-2']] = df_clean[col_map['Alpha-2']].str.upper()
    if 'Alpha-3' in col_map:
        df_clean[col_map['Alpha-3']] = df_clean[col_map['Alpha-3']].str.upper()
    
    # Convert Numeric to nullable int
    if 'Numeric' in col_map:
        df_clean[col_map['Numeric']] = pd.to_numeric(
            df_clean[col_map['Numeric']], 
            errors='coerce'
        ).astype('Int64')  # Nullable integer type
    
    # Rename columns to standard names
    rename_dict = {v: k for k, v in col_map.items()}
    df_clean = df_clean.rename(columns=rename_dict)
    
    # Drop rows with invalid data (no Country or no Numeric)
    if 'Country' in df_clean.columns and 'Numeric' in df_clean.columns:
        df_clean = df_clean.dropna(subset=['Country'])
        df_clean = df_clean[df_clean['Country'] != 'nan']
    
    return df_clean


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    """
    if 'Numeric' not in df.columns:
        raise ValueError("'Numeric' column not found in DataFrame")
    
    # Sort by Numeric in descending order and take top N
    df_sorted = df.sort_values('Numeric', ascending=False).head(top)
    return df_sorted


# --- Main execution ---

def main():
    # Fetch the webpage
    url = "https://www.iban.com/country-codes"
    print(f"Fetching data from: {url}")
    
    response = requests.get(url)
    response.raise_for_status()
    html = response.text
    
    # Step 1: Read table
    print("\n[1] Reading table from HTML...")
    df = q1_read_table(html)
    print(f"Found table with {len(df)} rows and {len(df.columns)} columns")
    print(f"Columns: {list(df.columns)}")
    
    # Step 2: Clean data
    print("\n[2] Cleaning data...")
    df_clean = q1_clean(df)
    print(f"After cleaning: {len(df_clean)} rows")
    
    # Step 3: Sort and get top 15
    print("\n[3] Sorting by Numeric (descending) and getting Top 15...")
    df_top15 = q1_sort_top(df_clean, top=15)
    
    # Step 4: Save to CSV
    output_file = "data_q1.csv"
    df_clean.to_csv(output_file, index=False)
    print(f"\n[4] Saved full data to: {output_file}")
    
    # Step 5: Display top 15
    print("\n" + "="*80)
    print("TOP 15 COUNTRIES BY NUMERIC CODE (Descending)")
    print("="*80)
    print(df_top15.to_string(index=False))
    print("="*80)
    
    return df_clean, df_top15


if __name__ == "__main__":
    df_full, df_top = main()

Fetching data from: https://www.iban.com/country-codes

[1] Reading table from HTML...
Found table with 249 rows and 4 columns
Columns: ['Country', 'Alpha-2 code', 'Alpha-3 code', 'Numeric']

[2] Cleaning data...
After cleaning: 249 rows

[3] Sorting by Numeric (descending) and getting Top 15...

[4] Saved full data to: data_q1.csv

TOP 15 COUNTRIES BY NUMERIC CODE (Descending)
                                                   Country Alpha-2 Alpha-3  Numeric
                                                    Zambia      ZM     ZMB      894
                                                     Yemen      YE     YEM      887
                                                     Samoa      WS     WSM      882
                                         Wallis and Futuna      WF     WLF      876
                        Venezuela (Bolivarian Republic of)      VE     VEN      862
                                                Uzbekistan      UZ     UZB      860
                               

/var/folders/2z/qxdng76n0yz7cvb65fsq52t00000gp/T/ipykernel_36871/4201773628.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

# --- Q2 Implementation ---

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
    rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, 'html.parser')
    
    items = []
    
    # Find all story rows (class 'athing')
    story_rows = soup.find_all('tr', class_='athing')
    
    for story_row in story_rows:
        # Get rank from the story row
        rank_span = story_row.find('span', class_='rank')
        rank = rank_span.get_text(strip=True).rstrip('.') if rank_span else ''
        
        # Get title and link from the story row
        title_span = story_row.find('span', class_='titleline')
        if title_span:
            title_link = title_span.find('a')
            title = title_link.get_text(strip=True) if title_link else ''
            link = title_link.get('href', '') if title_link else ''
        else:
            title = ''
            link = ''
        
        # Get the next sibling row which contains subtext (points, user, comments)
        subtext_row = story_row.find_next_sibling('tr')
        
        points = ''
        comments = ''
        user = ''
        
        if subtext_row:
            subtext = subtext_row.find('td', class_='subtext')
            if subtext:
                # Extract points
                score_span = subtext.find('span', class_='score')
                if score_span:
                    points_text = score_span.get_text(strip=True)
                    # Extract numeric value from "123 points"
                    points_match = re.search(r'(\d+)', points_text)
                    points = points_match.group(1) if points_match else ''
                
                # Extract user
                user_link = subtext.find('a', class_='hnuser')
                if user_link:
                    user = user_link.get_text(strip=True)
                
                # Extract comments count
                # Find all links and look for one containing 'comment'
                links = subtext.find_all('a')
                for a in links:
                    link_text = a.get_text(strip=True)
                    if 'comment' in link_text.lower():
                        # Extract number from "123 comments" or "discuss"
                        comments_match = re.search(r'(\d+)', link_text)
                        comments = comments_match.group(1) if comments_match else ''
                        break
        
        items.append({
            'rank': rank,
            'title': title,
            'link': link,
            'points': points,
            'comments': comments,
            'user': user
        })
    
    return pd.DataFrame(items)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    Cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    df_clean = df.copy()
    
    # Clean and convert numeric fields to int (non-digits -> 0)
    for col in ['rank', 'points', 'comments']:
        if col in df_clean.columns:
            # Convert to string first, extract digits only, convert to int
            df_clean[col] = df_clean[col].astype(str).apply(
                lambda x: int(re.sub(r'\D', '', x)) if re.sub(r'\D', '', x) else 0
            )
    
    # Fill missing text fields with empty string or appropriate defaults
    text_fields = ['title', 'link', 'user']
    for col in text_fields:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('').astype(str)
            # Replace 'nan' string with empty string
            df_clean[col] = df_clean[col].replace('nan', '')
    
    return df_clean


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    if 'points' not in df.columns:
        raise ValueError("'points' column not found in DataFrame")
    
    # Sort by points in descending order and take top N
    df_sorted = df.sort_values('points', ascending=False).head(top)
    return df_sorted


# --- Main execution ---

def main():
    # Fetch the webpage
    url = "https://news.ycombinator.com/"
    print(f"Fetching data from: {url}")
    
    response = requests.get(url)
    response.raise_for_status()
    html = response.text
    
    # Step 1: Parse items
    print("\n[1] Parsing Hacker News front page items...")
    df = q2_parse_items(html)
    print(f"Found {len(df)} stories")
    print(f"Columns: {list(df.columns)}")
    
    # Step 2: Clean data
    print("\n[2] Cleaning data...")
    df_clean = q2_clean(df)
    print(f"After cleaning: {len(df_clean)} rows")
    print(f"Sample data types:")
    print(df_clean.dtypes)
    
    # Step 3: Sort and get top 15
    print("\n[3] Sorting by points (descending) and getting Top 15...")
    df_top15 = q2_sort_top(df_clean, top=15)
    
    # Step 4: Save to CSV
    output_file = "data_q2.csv"
    df_clean.to_csv(output_file, index=False)
    print(f"\n[4] Saved full data to: {output_file}")
    
    # Step 5: Display top 15
    print("\n" + "="*100)
    print("TOP 15 HACKER NEWS STORIES BY POINTS (Descending)")
    print("="*100)
    
    # Set pandas display options for full output
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 60)
    
    # Format for better display
    display_df = df_top15[['rank', 'title', 'points', 'comments', 'user']].copy()
    
    # Print each row individually for guaranteed visibility
    print(f"\n{'Rank':<6} {'Title':<62} {'Points':<8} {'Comments':<10} {'User':<15}")
    print("-" * 100)
    
    for _, row in display_df.iterrows():
        title_short = row['title'][:58] + '...' if len(row['title']) > 58 else row['title']
        print(f"{row['rank']:<6} {title_short:<62} {row['points']:<8} {row['comments']:<10} {row['user']:<15}")
    
    print("="*100)
    
    # Show some statistics
    print(f"\nStatistics:")
    print(f"  Total stories: {len(df_clean)}")
    print(f"  Average points: {df_clean['points'].mean():.1f}")
    print(f"  Average comments: {df_clean['comments'].mean():.1f}")
    print(f"  Stories with 0 points: {(df_clean['points'] == 0).sum()}")
    
    return df_clean, df_top15


if __name__ == "__main__":
    df_full, df_top = main()

Fetching data from: https://news.ycombinator.com/

[1] Parsing Hacker News front page items...
Found 30 stories
Columns: ['rank', 'title', 'link', 'points', 'comments', 'user']

[2] Cleaning data...
After cleaning: 30 rows
Sample data types:
rank         int64
title       object
link        object
points       int64
comments     int64
user        object
dtype: object

[3] Sorting by points (descending) and getting Top 15...

[4] Saved full data to: data_q2.csv

TOP 15 HACKER NEWS STORIES BY POINTS (Descending)

Rank   Title                                                          Points   Comments   User           
----------------------------------------------------------------------------------------------------
30     Denmark's government aims to ban access to social media fo...  409      297        c420           
10     YouTube Removes Windows 11 Bypass Tutorials, Claims 'Risk ...  374      136        WaitWaitWha    
17     I Love OCaml                                             